### 导入包

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing 
from category_encoders import *
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score, precision_score
from sklearn.model_selection import RandomizedSearchCV

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn import datasets, linear_model, metrics
from sklearn.metrics import  confusion_matrix
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import BaggingClassifier
from mlxtend.classifier import StackingClassifier
from sklearn.datasets import make_classification
import warnings
warnings.filterwarnings("ignore")

### 导入数据、归一化、划分训练集测试集

In [3]:
data=pd.read_csv('./clean-data.csv')
data=data.drop(['Unnamed: 0'],axis=1)
X=data.drop(['Group'],axis=1)
y=data['Group']

In [4]:
column=X.columns
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X=sc.fit_transform(X)
X=pd.DataFrame(data=X,columns=column)

In [5]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=None)
print(f'Train shape : {X_train.shape}\nTest shape: {X_test.shape}')

Train shape : (132, 203)
Test shape: (33, 203)


### 交叉验证函数

In [6]:
def crosspredict(estimator,Xtrain,ytrain,cv):
    print("cross-validate across the entire data set")
    y_pred_cross=cross_val_predict(estimator,Xtrain,ytrain,cv=cv)
    confusion_cross=confusion_matrix(ytrain,y_pred_cross)
    a=accuracy_score(ytrain,y_pred_cross)
    p=precision_score(ytrain, y_pred_cross)
    r=recall_score(ytrain, y_pred_cross)
    f1=f1_score(ytrain, y_pred_cross)
    wf1=f1_score(ytrain, y_pred_cross, average='weighted')
    #auc=roc_auc_score(ytrain,estimator.predict_proba(Xtrain)[:,1])
    print('the confusion_matrix of the model is : \n',confusion_cross)
    print('the accuracy of the model is : ',a)
    print("the precision score of the model is : ", p)
    print("the recall score of the model is :", r)
    print('the f1_score of the model  is :',f1)
    print('the weighted_f1 of the model is :',wf1)
    print('the classification_report is :\n',classification_report(ytrain, y_pred_cross,digits=4))
    #print('the auc is :',auc)
    return a,p,r,f1,wf1

In [ ]:
clf1=SVC(C=1, coef0=10, degree=2, kernel='poly', probability=True)
clf2=KNeighborsClassifier(n_neighbors=13)
clf3=LogisticRegression(C=0.1)
clf4=GaussianNB()
clf5=BernoulliNB()
clf6=DecisionTreeClassifier(max_depth=5, min_samples_leaf=2, min_samples_split=6)
clf7=RandomForestClassifier(max_depth=8, n_estimators=105, n_jobs=-1,random_state=90)
clf8=AdaBoostClassifier(learning_rate=0.1)
clf9=GradientBoostingClassifier(loss='exponential', n_estimators=300)
clf10=XGBClassifier(booster='gblinear',learning_rate=0.3,n_estimators=10)
clf11=lgb.LGBMClassifier(max_depth=3, n_estimators=300)

In [43]:
sel_seq_dt_mask=['APG_0101010010M','APG_0101010055P','APG_0101010063M','APG_0101010341M','APG_0101010354P']
sel_seq_rf_mask=['APG_0101010055P','APG_0101010262P','APG_0101010266P']
sel_seq_ada_mask=['APG_0101010033M','APG_0101010063M','APG_0101010067M','APG_0101010147M','APG_0101010192M', 'APG_0101010236M']
sel_seq_gb_mask=['APG_0101010002M','APG_0101010055P','APG_0101010266P','APG_0101010354P']
sel_seq_xgb_mask=['APG_0101010010M','APG_0101010037P','APG_0101010063M','APG_0101010078P','APG_0101010150P','APG_0101010230P','APG_0101010279P','APG_0101010312P','APG_0101020013M']
sel_seq_KN_mask=['APG_0101010055P','APG_0101010100P','APG_0101010239P','APG_0101020013M']
sel_seq_Lightgbm_mask=['APG_0101010036M','APG_0101010078P','APG_0101010130P','APG_0101010192M','APG_0101010226P','APG_0101010242P','APG_0101010265P']
sel_seq_SVC_mask=['APG_0101010009P','APG_0101010055P','APG_0101010100P','APG_0101010143P','APG_0101010169P','APG_0101010199P','APG_0101010254P','APG_0101010267P', 'APG_0101010300P','APG_0101010356P']
sel_seq_GNB_mask=['APG_0101010010M','APG_0101010055P','APG_0101010057M','APG_0101010081M','APG_0101010085P','APG_0101010199P','APG_0101010264P','APG_0101010279P', 'APG_0101010300P']
sel_seq_lr_mask=['APG_0101010010M','APG_0101010030M','APG_0101010041M','APG_0101010043P','APG_0101010137P','APG_0101010150P','APG_0101010152M','APG_0101010168P','APG_0101010176P','APG_0101010356P']

model_dt_fs = MyEstimator(DecisionTreeClassifier(max_depth=5, min_samples_leaf=2, min_samples_split=6), sel_seq_dt_mask)
model_rf_fs = MyEstimator(RandomForestClassifier(max_depth=8, n_estimators=105, n_jobs=-1,random_state=90), sel_seq_rf_mask)
model_ada_fs = MyEstimator(AdaBoostClassifier(learning_rate=0.1), sel_seq_ada_mask)
model_gb_fs = MyEstimator(GradientBoostingClassifier(loss='exponential', n_estimators=300), sel_seq_gb_mask)
model_xgb_fs = MyEstimator(XGBClassifier(booster='gblinear',learning_rate=0.3,n_estimators=10), mask=sel_seq_xgb_mask)
model_KN_fs = MyEstimator(KNeighborsClassifier(n_neighbors=13), mask=sel_seq_KN_mask)
model_Lightgbm_fs = MyEstimator(lgb.LGBMClassifier(max_depth=3, n_estimators=300), mask=sel_seq_Lightgbm_mask)
model_SVC_fs = MyEstimator(SVC(C=1, coef0=10, degree=2, kernel='poly', probability=True),mask=sel_seq_SVC_mask)
model_GNB_fs = MyEstimator(GaussianNB(),mask=sel_seq_GNB_mask)
model_lr_fs = MyEstimator(LogisticRegression(C=0.1),mask=sel_seq_lr_mask)


In [33]:
a,b,c,d,e=crosspredict(model_dt_fs,X,y,10)

cross-validate across the entire data set
the confusion_matrix of the model is : 
 [[68 10]
 [25 62]]
the accuracy of the model is :  0.7878787878787878
the precision score of the model is :  0.8611111111111112
the recall score of the model is : 0.7126436781609196
the f1_score of the model  is : 0.779874213836478
the weighted_f1 of the model is : 0.7871766322610414
the classification_report is :
               precision    recall  f1-score   support

           0     0.7312    0.8718    0.7953        78
           1     0.8611    0.7126    0.7799        87

    accuracy                         0.7879       165
   macro avg     0.7961    0.7922    0.7876       165
weighted avg     0.7997    0.7879    0.7872       165



In [34]:
a,b,c,d,e=crosspredict(model_rf_fs,X,y,10)

cross-validate across the entire data set
the confusion_matrix of the model is : 
 [[64 14]
 [24 63]]
the accuracy of the model is :  0.7696969696969697
the precision score of the model is :  0.8181818181818182
the recall score of the model is : 0.7241379310344828
the f1_score of the model  is : 0.7682926829268294
the weighted_f1 of the model is : 0.7696123741084071
the classification_report is :
               precision    recall  f1-score   support

           0     0.7273    0.8205    0.7711        78
           1     0.8182    0.7241    0.7683        87

    accuracy                         0.7697       165
   macro avg     0.7727    0.7723    0.7697       165
weighted avg     0.7752    0.7697    0.7696       165



In [35]:
a,b,c,d,e=crosspredict(model_ada_fs,X,y,10)

cross-validate across the entire data set
the confusion_matrix of the model is : 
 [[56 22]
 [15 72]]
the accuracy of the model is :  0.7757575757575758
the precision score of the model is :  0.7659574468085106
the recall score of the model is : 0.8275862068965517
the f1_score of the model  is : 0.7955801104972374
the weighted_f1 of the model is : 0.7748263157362493
the classification_report is :
               precision    recall  f1-score   support

           0     0.7887    0.7179    0.7517        78
           1     0.7660    0.8276    0.7956        87

    accuracy                         0.7758       165
   macro avg     0.7773    0.7728    0.7736       165
weighted avg     0.7767    0.7758    0.7748       165



In [36]:
a,b,c,d,e=crosspredict(model_gb_fs,X,y,10)

cross-validate across the entire data set
the confusion_matrix of the model is : 
 [[64 14]
 [26 61]]
the accuracy of the model is :  0.7575757575757576
the precision score of the model is :  0.8133333333333334
the recall score of the model is : 0.7011494252873564
the f1_score of the model  is : 0.7530864197530864
the weighted_f1 of the model is : 0.7572550905884239
the classification_report is :
               precision    recall  f1-score   support

           0     0.7111    0.8205    0.7619        78
           1     0.8133    0.7011    0.7531        87

    accuracy                         0.7576       165
   macro avg     0.7622    0.7608    0.7575       165
weighted avg     0.7650    0.7576    0.7573       165



In [37]:
a,b,c,d,e=crosspredict(model_xgb_fs,X,y,10)

cross-validate across the entire data set
the confusion_matrix of the model is : 
 [[66 12]
 [30 57]]
the accuracy of the model is :  0.7454545454545455
the precision score of the model is :  0.8260869565217391
the recall score of the model is : 0.6551724137931034
the f1_score of the model  is : 0.7307692307692308
the weighted_f1 of the model is : 0.7439353749698577
the classification_report is :
               precision    recall  f1-score   support

           0     0.6875    0.8462    0.7586        78
           1     0.8261    0.6552    0.7308        87

    accuracy                         0.7455       165
   macro avg     0.7568    0.7507    0.7447       165
weighted avg     0.7606    0.7455    0.7439       165



In [38]:
a,b,c,d,e=crosspredict(model_KN_fs,X,y,10)

cross-validate across the entire data set
the confusion_matrix of the model is : 
 [[63 15]
 [43 44]]
the accuracy of the model is :  0.6484848484848484
the precision score of the model is :  0.7457627118644068
the recall score of the model is : 0.5057471264367817
the f1_score of the model  is : 0.6027397260273973
the weighted_f1 of the model is : 0.6415236341978451
the classification_report is :
               precision    recall  f1-score   support

           0     0.5943    0.8077    0.6848        78
           1     0.7458    0.5057    0.6027        87

    accuracy                         0.6485       165
   macro avg     0.6701    0.6567    0.6438       165
weighted avg     0.6742    0.6485    0.6415       165



In [39]:
a,b,c,d,e=crosspredict(model_Lightgbm_fs,X,y,10)

cross-validate across the entire data set
the confusion_matrix of the model is : 
 [[52 26]
 [24 63]]
the accuracy of the model is :  0.696969696969697
the precision score of the model is :  0.7078651685393258
the recall score of the model is : 0.7241379310344828
the f1_score of the model  is : 0.7159090909090908
the weighted_f1 of the model is : 0.6967237308146399
the classification_report is :
               precision    recall  f1-score   support

           0     0.6842    0.6667    0.6753        78
           1     0.7079    0.7241    0.7159        87

    accuracy                         0.6970       165
   macro avg     0.6960    0.6954    0.6956       165
weighted avg     0.6967    0.6970    0.6967       165



In [44]:
a,b,c,d,e=crosspredict(model_SVC_fs,X,y,10)

cross-validate across the entire data set
the confusion_matrix of the model is : 
 [[66 12]
 [30 57]]
the accuracy of the model is :  0.7454545454545455
the precision score of the model is :  0.8260869565217391
the recall score of the model is : 0.6551724137931034
the f1_score of the model  is : 0.7307692307692308
the weighted_f1 of the model is : 0.7439353749698577
the classification_report is :
               precision    recall  f1-score   support

           0     0.6875    0.8462    0.7586        78
           1     0.8261    0.6552    0.7308        87

    accuracy                         0.7455       165
   macro avg     0.7568    0.7507    0.7447       165
weighted avg     0.7606    0.7455    0.7439       165



In [41]:
a,b,c,d,e=crosspredict(model_GNB_fs,X,y,10)

cross-validate across the entire data set
the confusion_matrix of the model is : 
 [[75  3]
 [24 63]]
the accuracy of the model is :  0.8363636363636363
the precision score of the model is :  0.9545454545454546
the recall score of the model is : 0.7241379310344828
the f1_score of the model  is : 0.823529411764706
the weighted_f1 of the model is : 0.8348409317502039
the classification_report is :
               precision    recall  f1-score   support

           0     0.7576    0.9615    0.8475        78
           1     0.9545    0.7241    0.8235        87

    accuracy                         0.8364       165
   macro avg     0.8561    0.8428    0.8355       165
weighted avg     0.8614    0.8364    0.8348       165



In [42]:
a,b,c,d,e=crosspredict(model_lr_fs,X,y,10)

cross-validate across the entire data set
the confusion_matrix of the model is : 
 [[60 18]
 [22 65]]
the accuracy of the model is :  0.7575757575757576
the precision score of the model is :  0.7831325301204819
the recall score of the model is : 0.7471264367816092
the f1_score of the model  is : 0.7647058823529411
the weighted_f1 of the model is : 0.7577540106951872
the classification_report is :
               precision    recall  f1-score   support

           0     0.7317    0.7692    0.7500        78
           1     0.7831    0.7471    0.7647        87

    accuracy                         0.7576       165
   macro avg     0.7574    0.7582    0.7574       165
weighted avg     0.7588    0.7576    0.7578       165



### 集成

In [45]:
from sklearn.base import ClassifierMixin
from sklearn.base import TransformerMixin, BaseEstimator
class MyEstimator(BaseEstimator, ClassifierMixin):
    def __init__(self, model, mask) -> None:
        super().__init__()
        self.mask = mask
        self.model = model
    
    def fit(self, X, y=None):
        self.model.fit(X[self.mask],y)
        return self

    def predict(self, X):
        return self.model.predict(X[self.mask])

### Voting(dt/rf/ada/gb/xgb)

In [39]:
from itertools import *
sel_seq_dt_mask=['APG_0101010010M','APG_0101010055P','APG_0101010063M','APG_0101010341M','APG_0101010354P']
sel_seq_rf_mask=['APG_0101010055P','APG_0101010262P','APG_0101010266P']
sel_seq_ada_mask=['APG_0101010033M','APG_0101010063M','APG_0101010067M','APG_0101010147M','APG_0101010192M', 'APG_0101010236M']
sel_seq_gb_mask=['APG_0101010002M','APG_0101010055P','APG_0101010266P','APG_0101010354P']
sel_seq_xgb_mask=['APG_0101010055P', 'APG_0101010145P', 'APG_0101010176P']

feature=list(set(sel_seq_dt_mask+sel_seq_rf_mask+sel_seq_ada_mask+sel_seq_gb_mask+sel_seq_xgb_mask))
model_dt_fs = MyEstimator(DecisionTreeClassifier(max_depth=5, min_samples_leaf=2, min_samples_split=6), sel_seq_dt_mask)
model_rf_fs = MyEstimator(RandomForestClassifier(max_depth=8, n_estimators=105, n_jobs=-1,random_state=90), sel_seq_rf_mask)
model_ada_fs = MyEstimator(AdaBoostClassifier(learning_rate=0.1), sel_seq_ada_mask)
model_gb_fs = MyEstimator(GradientBoostingClassifier(loss='exponential', n_estimators=300), sel_seq_gb_mask)
model_xgb_fs = MyEstimator(XGBClassifier(booster='gbtree',learning_rate=0.01,n_estimators=100,max_depth=5), mask=sel_seq_xgb_mask)
from sklearn.ensemble import VotingClassifier
model_voting = VotingClassifier([
    ("dt",model_dt_fs),
    ("rf",model_rf_fs),
    ("ada", model_ada_fs),
    ("gb",model_gb_fs),
    ("xgb", model_xgb_fs)
], voting="hard")
a,b,c,d,e=crosspredict(model_voting,X,y,10)

cross-validate across the entire data set
the confusion_matrix of the model is : 
 [[73  5]
 [22 65]]
the accuracy of the model is :  0.8363636363636363
the precision score of the model is :  0.9285714285714286
the recall score of the model is : 0.7471264367816092
the f1_score of the model  is : 0.8280254777070064
the weighted_f1 of the model is : 0.835544279732638
the classification_report is :
               precision    recall  f1-score   support

           0     0.7684    0.9359    0.8439        78
           1     0.9286    0.7471    0.8280        87

    accuracy                         0.8364       165
   macro avg     0.8485    0.8415    0.8360       165
weighted avg     0.8529    0.8364    0.8355       165



### Voting(dt/rf/ada/xgb/gnb)

In [41]:
from itertools import *
sel_seq_dt_mask=['APG_0101010010M','APG_0101010055P','APG_0101010063M','APG_0101010341M','APG_0101010354P']
sel_seq_rf_mask=['APG_0101010055P','APG_0101010262P','APG_0101010266P']
sel_seq_ada_mask=['APG_0101010033M','APG_0101010063M','APG_0101010067M','APG_0101010147M','APG_0101010192M', 'APG_0101010236M']
sel_seq_gb_mask=['APG_0101010002M','APG_0101010055P','APG_0101010266P','APG_0101010354P']
sel_seq_xgb_mask=['APG_0101010010M','APG_0101010037P','APG_0101010063M','APG_0101010078P','APG_0101010150P','APG_0101010230P','APG_0101010279P','APG_0101010312P','APG_0101020013M']
sel_seq_gnb_mask=['APG_0101010010M','APG_0101010055P','APG_0101010057M','APG_0101010081M','APG_0101010085P','APG_0101010199P','APG_0101010264P','APG_0101010279P', 'APG_0101010300P']
feature=list(set(sel_seq_dt_mask+sel_seq_rf_mask+sel_seq_ada_mask+sel_seq_gb_mask+sel_seq_xgb_mask))
model_dt_fs = MyEstimator(DecisionTreeClassifier(max_depth=5, min_samples_leaf=2, min_samples_split=6), sel_seq_dt_mask)
model_rf_fs = MyEstimator(RandomForestClassifier(max_depth=8, n_estimators=105, n_jobs=-1,random_state=90), sel_seq_rf_mask)
model_ada_fs = MyEstimator(AdaBoostClassifier(learning_rate=0.1), sel_seq_ada_mask)
model_gb_fs = MyEstimator(GradientBoostingClassifier(loss='exponential', n_estimators=300), sel_seq_gb_mask)
model_xgb_fs = MyEstimator(XGBClassifier(booster='gblinear',learning_rate=0.3,n_estimators=10), mask=sel_seq_xgb_mask)
model_gnb_fs=MyEstimator(GaussianNB(),mask=sel_seq_gnb_mask)
from sklearn.ensemble import VotingClassifier
model_voting_hard = VotingClassifier([
    ("dt",model_dt_fs),
    ("rf",model_rf_fs),
    ("ada", model_ada_fs),
    ("xgb", model_xgb_fs),
    ("gnb",model_gnb_fs),
], voting="hard")
a,b,c,d,e=crosspredict(model_voting_hard,X,y,10)

cross-validate across the entire data set
the confusion_matrix of the model is : 
 [[76  2]
 [16 71]]
the accuracy of the model is :  0.8909090909090909
the precision score of the model is :  0.9726027397260274
the recall score of the model is : 0.8160919540229885
the f1_score of the model  is : 0.8875
the weighted_f1 of the model is : 0.8906283422459892
the classification_report is :
               precision    recall  f1-score   support

           0     0.8261    0.9744    0.8941        78
           1     0.9726    0.8161    0.8875        87

    accuracy                         0.8909       165
   macro avg     0.8993    0.8952    0.8908       165
weighted avg     0.9033    0.8909    0.8906       165



In [48]:
from sklearn.ensemble import VotingClassifier
model_voting_hard = VotingClassifier([
    ("dt",model_dt_fs),
    ("rf",model_rf_fs),
    ("ada", model_ada_fs),
    ("xgb", model_xgb_fs),
    ("gnb",model_GNB_fs),
], voting="hard")
a,b,c,d,e=crosspredict(model_voting_hard,X,y,10)

cross-validate across the entire data set
the confusion_matrix of the model is : 
 [[76  2]
 [16 71]]
the accuracy of the model is :  0.8909090909090909
the precision score of the model is :  0.9726027397260274
the recall score of the model is : 0.8160919540229885
the f1_score of the model  is : 0.8875
the weighted_f1 of the model is : 0.8906283422459892
the classification_report is :
               precision    recall  f1-score   support

           0     0.8261    0.9744    0.8941        78
           1     0.9726    0.8161    0.8875        87

    accuracy                         0.8909       165
   macro avg     0.8993    0.8952    0.8908       165
weighted avg     0.9033    0.8909    0.8906       165



In [49]:
from mlxtend.classifier import StackingClassifier
model_Stacking_cross = StackingClassifier(classifiers=[
    (model_dt_fs),
    (model_rf_fs ),
    (model_ada_fs),
    (model_GNB_fs),
    (model_xgb_fs),
],meta_classifier=LogisticRegression(C=0.1))
a,b,c,d,e=crosspredict(model_Stacking_cross,X,y,10)

cross-validate across the entire data set
the confusion_matrix of the model is : 
 [[72  6]
 [12 75]]
the accuracy of the model is :  0.8909090909090909
the precision score of the model is :  0.9259259259259259
the recall score of the model is : 0.8620689655172413
the f1_score of the model  is : 0.8928571428571429
the weighted_f1 of the model is : 0.890981240981241
the classification_report is :
               precision    recall  f1-score   support

           0     0.8571    0.9231    0.8889        78
           1     0.9259    0.8621    0.8929        87

    accuracy                         0.8909       165
   macro avg     0.8915    0.8926    0.8909       165
weighted avg     0.8934    0.8909    0.8910       165



Stacking 1D_CNN 5个基模型 第二层用神经网络NN

In [52]:
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D, Input, concatenate
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler


# 对特征进行标准化处理
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 训练LASSO回归模型，并获取系数
lasso = Lasso(alpha=0.1)
lasso.fit(X, y)
coefs = lasso.coef_

# 选择系数不为0的特征
selected_features = X[:, coefs != 0]
X=selected_features
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=None)
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
for train_idx, test_idx in kfold.split(X, y):
    X_train = X[train_idx]
    y_train = y[train_idx]
    X_test = X[test_idx]
    y_test = y[test_idx]
    
    models = []
    
    for i in range(5):
        model = Sequential()
        model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dense(1, activation='softmax'))

    first_layer_train = np.zeros((len(X_train), 5))
    first_layer_test = np.zeros((len(X_test), 5))

    for i, model in enumerate(models):
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.fit(X_train, y_train, epochs=50, batch_size=32)

        y_pred_train = model.predict(X_train)
        first_layer_train[:, i] = y_pred_train.ravel()

        y_pred_test = model.predict(X_test)
        first_layer_test[:, i] = y_pred_test.ravel()

    input_layer = Input(shape=(5,))
    x = Dense(32, activation='relu')(input_layer)
#     x = Dropout(0.5)(x)
    output_layer = Dense(1, activation='sigmoid')(x)
    nn_model = Model(inputs=input_layer, outputs=output_layer)

    nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    nn_model.fit(first_layer_train, y_train, epochs=50, batch_size=32)

    y_pred_nn = nn_model.predict(first_layer_test)

    y_pred_nn_binary = np.round(y_pred_nn)
    accuracy_scores.append(accuracy_score(y_test, y_pred_nn_binary))
    precision_scores.append(precision_score(y_test, y_pred_nn_binary))
    recall_scores.append(recall_score(y_test, y_pred_nn_binary))
    f1_scores.append(f1_score(y_test, y_pred_nn_binary))

print('Accuracy:', np.mean(accuracy_scores))
print('Precision:', np.mean(precision_scores))
print('Recall:', np.mean(recall_scores))
print('F1 Score:', np.mean(f1_scores))

Accuracy: 0.56859243697479 Precision: 0.5735693263319314 Recall: 0.8880952380952382 F1 Score: 0.683120473257446